## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 4.4 MB 4.4 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 
     |████████████████████████████████| 86 kB 1.1 MB/s 
     |████████████████████████████████| 596 kB 35.9 MB/s 
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


## **Importing Libraries**

In [4]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from imblearn.over_sampling import SMOTE
from collections import Counter

from transformers import AutoTokenizer, AutoModel

import pickle

## **Setting up Stemmer**

In [5]:
! rm -r tamil-stemmer-build
! rm input.txt
! rm output.txt

! mkdir tamil-stemmer-build
% cd tamil-stemmer-build
! git clone https://github.com/rdamodharan/tamil-stemmer.git
! tar -zxf tamil-stemmer/snowball-with-tamil.tgz
% cd snowball-with-tamil
! make

% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build
Cloning into 'tamil-stemmer'...
remote: Enumerating objects: 88, done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (88/88), done.
/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build/snowball-with-tamil
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/space.o compiler/space.c
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/tokeniser.o compiler/tokeniser.c
In file included from compiler/tokeniser.c:6:0:
compiler/tokeniser.c: In function ‘read_token’:
compiler/header.h:8:21: warning: this statement may fall through [-Wimplicit-fallthrough=]
 #define unless(C) if(!(C))
                     ^
compiler/tokeniser.c:390:16: note: in expansion of macro ‘unless’
                unless (t->next == 0) {
               

## **Loading Data**

In [6]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [7]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [8]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [9]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [10]:
dataframe.shape

(5880, 3)

In [11]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [12]:
dataframe.shape

(5877, 3)

In [13]:
text = dataframe['text']

In [14]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [15]:
label = dataframe['category']

In [16]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [17]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [18]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [19]:
text = text.tolist()

! rm input.txt
! rm output.txt

for i in text:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [20]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [21]:
text_file = open("output.txt", "r")

text = []

for line in text_file:
    text.append(line.strip())

In [22]:
# text = np.array(text)
# text

## **Feature Extraction**

In [23]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/tamillion")
model = AutoModel.from_pretrained("monsoon-nlp/tamillion")

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/817k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at monsoon-nlp/tamillion were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [25]:
print(tokenized_input)

{'input_ids': tensor([[    3, 12190,  9946, 12372,  6651,  5052,  3322,  2599,  2686,     4]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [26]:
print(sample_output)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         ...,
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05]]], grad_fn=<NativeLayerNormBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)


In [27]:
print(sample_output.last_hidden_state.cpu().detach().numpy().shape)

(1, 10, 768)


In [28]:
def max_pool(model_output):
    pooled_output = []
    for i in range(len(model_output[0][0])):
        max = model_output[0][0][i]
        for j in range(len(model_output[0])):
            if max < model_output[0][j][i]:
                max = model_output[0][j][i]
        pooled_output.append(max)
    pooled_output = np.array(pooled_output)
    pooled_output = pooled_output.reshape(1, len(pooled_output))
    return pooled_output

In [29]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [30]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.last_hidden_state.cpu().detach().numpy()
    output.append(max_pool(model_output))
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output1.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output2.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output3.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output4.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output5.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output6.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output7.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output8.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output9.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output10.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output11.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output12.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output13.pickle 

In [31]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output1.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output2.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output3.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output4.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output5.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output6.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output7.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output8.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output9.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output10.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output11.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output12.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TrainData/output13.pickle 

In [32]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [33]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [34]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
X_train.shape

(4701, 768)

In [37]:
X_val.shape

(1176, 768)

## **SMOTE Oversampling**

In [38]:
counter = Counter(y_train)
print(counter)

Counter({0: 3774, 1: 927})


In [39]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [40]:
counter = Counter(y_train)
print(counter)

Counter({0: 3774, 1: 3774})


## **Logistic Regression**

In [41]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       950
           1       0.19      1.00      0.32       226

    accuracy                           0.19      1176
   macro avg       0.10      0.50      0.16      1176
weighted avg       0.04      0.19      0.06      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Support Vector Machine**

In [42]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.83      0.29      0.43       950
           1       0.20      0.76      0.32       226

    accuracy                           0.38      1176
   macro avg       0.52      0.52      0.37      1176
weighted avg       0.71      0.38      0.41      1176



## **Naive Bayes**

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

## **Stochastic Gradient Descent**

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

## **K Nearest Neighbours**

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

## **Decision tree**

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

## **Random Forest**

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

## **Majority Voting**

In [ ]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Keeping only DT, KNN & NB in predictions due to poor performance
    predictions = [dt_pred[i], knn_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

## **Loading Testing Data**

In [ ]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

In [ ]:
dataframe_test.describe()

In [ ]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [ ]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

In [ ]:
text_test = text_test.tolist()

! rm input.txt
! rm output.txt

for i in text_test:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

In [ ]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [55]:
text_file = open("output.txt", "r")

text_test = []

for line in text_file:
    text_test.append(line.strip())

text_test = np.array(text_test)

## **Test Feature Extraction**

In [56]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/tamillion")
model = AutoModel.from_pretrained("monsoon-nlp/tamillion")

Some weights of the model checkpoint at monsoon-nlp/tamillion were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [58]:
print(tokenized_input)

{'input_ids': tensor([[    3,  8689,  1017, 36488,  2008,  1961,  5222,  3230, 11190, 14066,
          1840, 33480,  8689,  1017,  7187,  8372,  1054,    71,  1041,  1073,
         30637,     4]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [59]:
print(sample_output)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         ...,
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05],
         [-8.7158e-04, -1.5213e-04,  1.6477e+00,  ..., -1.5470e-03,
          -2.9372e-05,  9.1286e-05]]], grad_fn=<NativeLayerNormBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)


In [60]:
print(sample_output.last_hidden_state.cpu().detach().numpy().shape)

(1, 22, 768)


In [61]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [62]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.last_hidden_state.cpu().detach().numpy()
    output.append(max_pool(model_output))
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TaMillion/TaMillion-NOStopword-Stem-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TaMillion/TaMillion-NOStopword-Stem-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TaMillion/TaMillion-NOStopword-Stem-Embedded-TestData/output1.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TestData/output2.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TestData/output3.pickle done
./TaMillion/TaMillion-NOStopword-Stem-Embedded-TestData/output4.pickle done


In [ ]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./TaMillion/TaMillion-NOStopword-Stem-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

In [ ]:
X_test = np.array(X_test)
X_test.shape

In [ ]:
X_test = X_test.reshape(654, 768)
X_test.shape

## **Getting Predictions**

In [ ]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [ ]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [rf_test_pred[i], knn_test_pred[i], dt_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [ ]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [ ]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [ ]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

In [ ]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

In [ ]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

In [ ]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

In [ ]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

In [ ]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

In [ ]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

In [77]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.88      0.57      0.69       536
           1       0.25      0.66      0.36       118

    accuracy                           0.58       654
   macro avg       0.57      0.61      0.53       654
weighted avg       0.77      0.58      0.63       654

Majority Voting
              precision    recall  f1-score   support

           0       0.88      0.57      0.69       536
           1       0.25      0.66      0.36       118

    accuracy                           0.58       654
   macro avg       0.57      0.61      0.53       654
weighted avg       0.77      0.58      0.63       654

